Gets the training data for the ML model.
Be sure to specify the correct paths to the directories, the API key for bing maps, and the regions for Bing Maps/Overpass Turbo.

NOTE: Ensure 'requests' is installed for Python3 by running ```pip install requests``` in your machine's terminal.

In [1]:
# Get every mapped feature from a specified boundary box (bbox) from Overpass Turbo
import requests
import json

# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Your Overpass query
overpass_query = """
[out:json][timeout:100];
// gather results
(
  // query part for: “parking=surface”
  node["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
  way["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
  relation["parking"="surface"](39.7042,-86.3994,39.9569,-86.0172);
);
// print results
out body;
>;
out skel qt;
"""

# Send the request
overpass_response = requests.get(overpass_url, params={'data': overpass_query})

# Check for HTTP errors
overpass_response.raise_for_status()

# Parse the JSON response
data = overpass_response.json()

# Save the data to a file
with open('data.json', 'w') as file:
    json.dump(data, file, indent=4)

In [2]:
# Method to process the Overpass API Query
def process_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Create a dictionary to map node IDs to their lat/lon
    node_dict = {}
    for element in data['elements']:
        if element['type'] == 'node':
            node_dict[element['id']] = (element['lat'], element['lon'])

    output_data = {}
    for element in data['elements']:
        if element['type'] == 'way':
            way_id = str(element['id'])
            node_ids = element['nodes']

            # Initialize min and max values
            min_lat = min_lon = float('inf')
            max_lat = max_lon = float('-inf')
            total_lat = total_lon = 0
            coordinate_list = []

            for node_id in node_ids:
                if node_id in node_dict:
                    lat, lon = node_dict[node_id]
                    min_lat = min(min_lat, lat)
                    max_lat = max(max_lat, lat)
                    min_lon = min(min_lon, lon)
                    max_lon = max(max_lon, lon)
                    total_lat += lat
                    total_lon += lon
                    coordinate_list.append(f"{lat},{lon}")

            center_lat = total_lat / len(node_ids)
            center_lon = total_lon / len(node_ids)
            coordinates = "_".join(coordinate_list)

            output_data[way_id] = {
                "min_lat": min_lat,
                "min_lon": min_lon,
                "max_lat": max_lat,
                "max_lon": max_lon,
                "center_point_lat": center_lat,
                "center_point_lon": center_lon,
                "coordinates": coordinates
            }

    return output_data

# Store cleaned data in data.json
processed_data = process_json('data.json')

# To save the processed data to a new file
with open('coordinates.json', 'w') as file:
    json.dump(processed_data, file, indent=4)

{
    "40006669": {
        "min_lat": 39.7085272,
        "min_lon": -86.3062743,
        "max_lat": 39.7129114,
        "max_lon": -86.2999529,
        "center_point_lat": 39.710593845541396,
        "center_point_lon": -86.30345059171981,
        "coordinates": "39.7107361,-86.3055576_39.710773,-86.3055096_39.7108442,-86.305417_39.7108846,-86.3053645_39.7108987,-86.3053506_39.7109104,-86.3053456_39.7109211,-86.3053473_39.7109301,-86.3053539_39.7109363,-86.3053645_39.7109384,-86.3053798_39.7109369,-86.3053962_39.7109278,-86.3054091_39.7108872,-86.3054524_39.7108626,-86.3054865_39.7108273,-86.3055388_39.7107983,-86.3055877_39.7107835,-86.3056241_39.7107715,-86.305663_39.7107655,-86.3057046_39.7107625,-86.3057469_39.7107662,-86.3057875_39.7107732,-86.3058333_39.7107849,-86.3058742_39.7108037,-86.3059128_39.7108286,-86.3059554_39.7109216,-86.3060766_39.7110732,-86.3062743_39.7111134,-86.3062173_39.7111472,-86.3061687_39.7110972,-86.306101_39.7110509,-86.3060383_39.7110575,-86.3060297_39